**Hyper columns**

Many algorithms using features from CNNs (Convolutional Neural Networks) usually use the last FC (fully-connected) layer features in order to extract information about certain input. However, the information in the last FC layer may be too coarse spatially to allow precise localization (due to sequences of maxpooling, etc.), on the other side, the first layers may be spatially precise but will lack semantic information. To get the best of both worlds, the authors of the hypercolumn paper define the hypercolumn of a pixel as the vector of activations of all CNN units “above” that pixel.
http://arxiv.org/pdf/1411.5752v2.pdf

![img](http://blog.christianperone.com/wp-content/uploads/2016/01/hypercolumn.png)

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")
import scipy.misc

In [ ]:
model = VGG16(weights='imagenet')

In [ ]:
img_path = "../input/train/images/1bd1c8c771.png"
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)

In [ ]:

plt.plot(preds.ravel())

In [ ]:
#https://keras.io/getting-started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer
from keras import backend as K
from keras.preprocessing.image import array_to_img, img_to_array, load_img
# with a Sequential model
get_feature = K.function([model.layers[0].input],
                                  [model.layers[2].output])
#layer_output = get_feature([x])[0]
feat = get_feature([x])[0]
print(feat.shape)


In [ ]:
conv_outputs = feat[0, :, :, :]
conv_outputs.shape

In [ ]:
feat_map = conv_outputs.transpose((2,0,1))
feat_map.shape

In [ ]:
#showing 3rd feature map
plt.imshow(feat_map[2])

**Extracting hypercolumns**

In [ ]:
from skimage.transform import resize
def extract_hypercolumn(model, layer_indexes, instance):
    layers = [model.layers[li].output for li in layer_indexes]
    get_feature = K.function([model.layers[0].input],layers)
    for layer in layers:
        print(layer.name)

    feature_maps = get_feature(instance)
    hypercolumns = []
    for convmap in feature_maps:
        print(convmap.shape)
        conv_out = convmap[0, :, :, :]
        feat_map = conv_out.transpose((2,0,1))
        print("F sz :",feat_map.shape)
        for fmap in feat_map: 
            #print(fmap.shape)
            upscaled =resize(fmap, (224, 224), mode='constant', preserve_range=True)
            hypercolumns.append(upscaled)
    return np.asarray(hypercolumns)

In [ ]:
for idx, layer in enumerate(model.layers):
    if "conv" in layer.name:
        print(idx, layer.name)    

In [ ]:
layers_extract = [1]
hc = extract_hypercolumn(model, layers_extract, [x])
print(hc.shape)

In [ ]:
avg_hc=np.average(hc, axis=0)
print(avg_hc.shape)
plt.imshow(avg_hc)


In [ ]:
layers_extract = [5,8,12]
hc2 = extract_hypercolumn(model, layers_extract, [x])

In [ ]:
import sklearn.cluster as cluster
m = hc2.transpose(1,2,0).reshape(224*224, -1)
kmeans = cluster.KMeans(n_clusters=2, max_iter=300, n_jobs=5, precompute_distances=True)
cluster_labels = kmeans .fit_predict(m)
imcluster = np.zeros((224,224))
imcluster = imcluster.reshape((224*224,))
imcluster = cluster_labels
plt.imshow(imcluster.reshape(224, 224), cmap="hot")